In [10]:
import pandas as pd
import numpy as np
import FinanceDataReader as fdr

## per, roa로 sort하는 함수(per은 낮을수록 좋고, roa는 높을수록 좋음)
def sort_val(s_val, asc = True, standard = 0):
  # s_val: 정렬할 series
  # asc: 오름차순 정렬 여부
  # standard: 기준값(기본값 0)
  
  s_val_mask = s_val.mask(s_val < standard, np.nan)
  s_val_mask_rank = s_val_mask.rank(ascending = asc, na_option = 'bottom')
  
  return s_val_mask_rank

df = pd.read_csv('../data/krx/per_roa.csv')
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)]

per = pd.to_numeric(df['PER'])
roa = pd.to_numeric(df['ROA'])

per_rank = sort_val(per, asc = True, standard = 0)
roa_rank = sort_val(roa, asc = False, standard = 0)

## 합산 랭킹
result_rank = per_rank + roa_rank
result_rank = sort_val(result_rank, asc = True)
result_rank = result_rank.where(result_rank <= 20, 0) # 상위 10개 종목 선정
result_rank = result_rank.mask(result_rank > 0, 1)

mf_df = df.loc[result_rank > 0, ['Code', 'Name', 'Marcap']].copy()
mf_df['Marcap'] = round(mf_df['Marcap'] / 100000000)
mf_stock_list = df.loc[result_rank > 0, 'Code']

for idx, code in enumerate(mf_df['Code'].values):
  code_name = mf_df.loc[mf_df['Code'] == code, 'Name'].values[0]
  print(code, code_name)
  df = fdr.DataReader(code, '2023-01-01', '2023-11-22')
  if idx == 0:
    mf_df_rtn = pd.DataFrame(index=df.index)
  df['daily_rtn'] = df['Close'].pct_change(periods=1)
  df['cum_rtn'] = (1 + df['daily_rtn']).cumprod()
  tmp = df.loc[:, ['cum_rtn']].rename(columns={'cum_rtn': code_name})
  
  mf_df_rtn = mf_df_rtn.join(tmp, how='left')
  df = None

print(mf_df_rtn.tail(5))

010060 OCI홀딩스
005440 현대지에프홀딩스
306200 세아제강
078150 HB테크놀러지
073570 리튬포어스
005860 한일사료
008260 NI스틸
016710 대성홀딩스
033920 무학
092460 한라IMS
208140 정다운
038010 제일테크노스
088910 동우팜투테이블
227950 엔투텍
094840 슈프리마에이치큐
139050 시티랩스
031510 오스템
089140 넥스턴바이오
043710 서울리거
017250 인터엠
              OCI홀딩스  현대지에프홀딩스      세아제강   HB테크놀러지     리튬포어스      한일사료  \
Date                                                                     
2023-11-16  1.632212  0.842814  0.992248  1.326316  0.496774  1.115152   
2023-11-17  1.662826  0.848619  0.991473  1.260526  0.465161  1.088485   
2023-11-20  1.661215  0.863710  0.992248  1.265789  0.478065  1.098182   
2023-11-21  1.645103  0.882285  1.016279  1.271053  0.470968  1.100606   
2023-11-22  1.614489  0.893894  1.027907  1.268421  0.457419  1.095758   

                NI스틸     대성홀딩스        무학     한라IMS       정다운    제일테크노스  \
Date                                                                     
2023-11-16  1.351536  0.090578  1.160209  1.114783  1.382075  1.165414   
20